# 1. 공시 사업 보고서 및 로드 전처리

In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL" -O dart.csv && rm -rf /tmp/cookies.txt

--2023-05-09 18:59:38--  https://docs.google.com/uc?export=download&confirm=t&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL
Resolving docs.google.com (docs.google.com)... 173.194.215.113, 173.194.215.101, 173.194.215.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.215.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tv7916r1kiievtbjbfgudtfgs5sh48u6/1683658725000/17609157229046208934/*/1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL?e=download&uuid=fd2d9920-c7f7-44da-baba-94666c5bd358 [following]
--2023-05-09 18:59:38--  https://doc-04-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tv7916r1kiievtbjbfgudtfgs5sh48u6/1683658725000/17609157229046208934/*/1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL?e=download&uuid=fd2d9920-c7f7-44da-baba-94666c5bd358
Resolving doc-04-7k-docs.googleusercontent.com (doc-04-7k-docs.googleusercontent.com)... 74.125.134.1

In [2]:
# 형태소 분석기 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 122 (delta 15), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (122/122), 1.27 MiB | 2.50 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 27.3 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-05-09 18:59:54--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.0, 18.205.93.2, 18.205.93.1, ...
Connecting to b

In [3]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install mecab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mecab: filename=mecab-0.996.3-cp310-cp310-linux_x86_64.whl size=185705 sha256=dd7d41516449ad1638761816ca04df669c128f59b2fb3444f42ca79c24c85c6c
  Stored in directory: /root/.cache/pip/wheels/10/1a/b2/8c0e17d02a12c7b46e3065f55aa7555d173a5b020110717829
Successfully built mecab


In [5]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

data.csv 파일을 데이터 프레임으로 로드. 결측값 가진 샘플 제거

In [6]:
df = pd.read_csv('/content/dart.csv',  sep=',')
df = df.dropna()
df

,code,market,name,business
0,000020,KOSPI,동화약품,II. 사업의 내용\n1. 사업의 개요\n가. 일반적인 사항\n기업회계기준서 제11...
1,000040,KOSPI,KR모터스,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n수출주력시장인 유럽 불...
2,000050,KOSPI,경방,II. 사업의 내용\n1. 사업의 개요\n(1) 산업의 특성\n[섬유사업부문]\n면...
3,000060,KOSPI,메리츠화재,"II. 사업의 내용\n1. 사업의 개요\n가. 산업의 특성, 성장성, 경기변동의 특..."
4,000070,KOSPI,삼양홀딩스,"II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n지주회사(持株會社, H..."
...,...,...,...,...
2582,950130,KOSDAQ,엑세스바이오,II. 사업의 내용\n1. 사업의 개\n요\n당사는 체외진단 기술을 토대로 말라리아...
2583,950140,KOSDAQ,잉글우드랩,II. 사업의 내용\n1. 사업의 개요\n가. 화장품 ODM산업 개요\nODM은 O...
2584,950160,KOSDAQ,코오롱티슈진,II. 사업의 내용\n당사는 1999년 6월 9일 세포 유전자 기술 Platform...
2585,950170,KOSDAQ,JTC,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n(1) 산업의 특성\n...


총 2,295개의 행과 4개의 열로 구성. 첫번째 열은 종목 번호헤 해당하는 code열, 두번째는 종목 KOSPI인지 KOSDAQ인지 알려주는 market, 세번째는 회사명 name, 네번째는 사업 보고서 business에 해당한다 

Business 열에 형태서 분석 수행. Doc2Vec 학습을 위해서는 해당 문서의 '제목'과 단어 토큰화가 된 상태의 해당 문서의 '본문' 이 두가지를 필요로 한다.

TaggedDocument의 tags에 해당 문서의 제목, 그리고 words에 본문의 토큰화를 저장. 이 결과를 원소로 하는 파이썬 리스트 tagged_corpus_list를 만든다

In [9]:
mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['business']
  tag = row['name']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list))

100%|██████████| 2295/2295 [03:00<00:00, 12.74it/s]

문서의 수 : 2295


In [11]:
tagged_corpus_list[0] # 첫번째 문서의 전처리 결과

TaggedDocument(words=['II', '.', '사업', '의', '내용', '1', '.', '사업', '의', '개요', '가', '.', '일반', '적', '인', '사항', '기업', '회계', '기준', '서', '제', '1110', '호', '"', '연결', '재무제표', '"', '의', '의하', '여', '2018', '년', '12', '월', '17', '일', '에', '설립', '한', '동화', '크립톤', '기업가', '정신', '제일', '호', '창업', '벤처', '전문', '사모', '투자', '합자회사', '를', '종속', '회사', '에', '편입', '하', '였', '습니다', '.', '나', '.', '지배', '기업', '의', '현황', '1', ')', '산업', '의', '특성', '제약', '산업', '은', '약학', ',', '화학', ',', '생물학', '등', '여러', '분야', '의', '지식', '과', '기술', '이', '복합', '적', '으로', '필요', '한', '기술', '·', '지식', '집약', '형', '산업', '으로', '인간', '의', '생명', '과', '보건', '에', '직접', '적', '으로', '관련', '된', '제품', '을', '생산', '하', '고', ',', '개발', '·', '제조', '·', '유통', '·', '가격', '등', '을', '정부', '에서', '엄격', '하', '게', '규제', '하', '고', '관리', '하', '는', '산업', '적', '특성', '을', '가지', '고', '있', '습니다', '.', '또한', '신약', '개발', '에', '는', '막대', '한', '시간', '과', '비용', '이', '소요', '되', '며', ',', '성공', '확률', '또한', '높', '지', '않', '으나', '신약', '개발', '에', '성공', '할', '경우', '엔', '엄청난

# 2. Doc2Vec 학습 및 테스트

In [13]:
from gensim.models import doc2vec

model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')

이제 3개의 파일이 생겼다

*   dart.doc2vec
*   dart.doc2vec.trainables.syn1neg.npy
*   dart.doc2vect.wv.vectors.npy



In [14]:
similar_doc = model.docvecs.most_similar('동화약품')
print(similar_doc)
# 모델 테스트, 동화약품과 사업 보고서가 유사한 회사들

[('종근당', 0.5528733134269714), ('동아에스티', 0.5040166974067688), ('영진약품', 0.5027133226394653), ('일양약품', 0.497097373008728), ('국제약품', 0.4925762712955475), ('제일약품', 0.4901142418384552), ('삼일제약', 0.48920488357543945), ('대웅제약', 0.4764939844608307), ('현대약품', 0.4762733578681946), ('비씨월드제약', 0.4683372676372528)]


<ipython-input-14-c3986a1e805b>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar('동화약품')


# 09-14 실전. 한국어 위키피디아로 Word2Vec 학습

## 1. 위키피디아로부터 데이터 다운로드 및 통합

In [15]:
pip install wikiextractor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.8 MB/s eta 0:00:00


In [16]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 122 (delta 15), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (122/122), 1.27 MiB | 3.51 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-05-09 20:00:18--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.2, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuserup

In [17]:
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2

--2023-05-09 20:00:59--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 900873558 (859M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles.xml.bz2’

kowiki-latest-pages 100%[===================>] 859.14M  4.20MB/s    in 3m 41s  

2023-05-09 20:04:40 (3.89 MB/s) - ‘kowiki-latest-pages-articles.xml.bz2’ saved [900873558/900873558]



위키피디 익스트랙터사용해서 덤프 파싱

In [18]:
!python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2

INFO: Preprocessing 'kowiki-latest-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Loaded 62722 templates in 430.7s
INFO: Starting page extraction from kowiki-latest-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Extracted 100000 articles (585.9 art/s)
INFO: Extracted 200000 articles (857.7 art/s)
INFO: Extracted 300000 articles (982.7 art/s)
INFO: Extracte

현재 경로에 있는 디렉토리와 파일들의 리스트 받아온다

In [19]:
%ls

images/                                    LICENSE
install_mecab-ko_on_colab190912.sh         README.md
install_mecab-ko_on_colab_light_220429.sh  text/
kowiki-latest-pages-articles.xml.bz2


text 디렉토리 안에 어떤 디렉토리 있는지 확인

In [20]:
import os
import re
os.listdir('text')

['AI', 'AB', 'AE', 'AH', 'AJ', 'AF', 'AA', 'AD', 'AG', 'AC']

AA라는 디렉토리 파일들 확인

In [21]:
%ls text/AA

wiki_00  wiki_12  wiki_24  wiki_36  wiki_48  wiki_60  wiki_72  wiki_84  wiki_96
wiki_01  wiki_13  wiki_25  wiki_37  wiki_49  wiki_61  wiki_73  wiki_85  wiki_97
wiki_02  wiki_14  wiki_26  wiki_38  wiki_50  wiki_62  wiki_74  wiki_86  wiki_98
wiki_03  wiki_15  wiki_27  wiki_39  wiki_51  wiki_63  wiki_75  wiki_87  wiki_99
wiki_04  wiki_16  wiki_28  wiki_40  wiki_52  wiki_64  wiki_76  wiki_88
wiki_05  wiki_17  wiki_29  wiki_41  wiki_53  wiki_65  wiki_77  wiki_89
wiki_06  wiki_18  wiki_30  wiki_42  wiki_54  wiki_66  wiki_78  wiki_90
wiki_07  wiki_19  wiki_31  wiki_43  wiki_55  wiki_67  wiki_79  wiki_91
wiki_08  wiki_20  wiki_32  wiki_44  wiki_56  wiki_68  wiki_80  wiki_92
wiki_09  wiki_21  wiki_33  wiki_45  wiki_57  wiki_69  wiki_81  wiki_93
wiki_10  wiki_22  wiki_34  wiki_46  wiki_58  wiki_70  wiki_82  wiki_94
wiki_11  wiki_23  wiki_35  wiki_47  wiki_59  wiki_71  wiki_83  wiki_95


AA 디렉토리의 wiki_00 파일

In [23]:
!cat text/AA/wiki_00

<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">
지미 카터

제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국의 제39대 대통령(1977년 ~ 1981년) 이다.
생애.
어린 시절.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
정계 입문.
1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
대통령 재임.
1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 대통령을 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
외교 정책.
카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.
소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들

10개 디렉토리 안의 wiki 숫자 형태의 수많은 파일들을 하나로 통합하는 과정을 진행. 모든 파일들의 경로를 파이썬의 리스트 형태로 저장합니다.

In [26]:
def list_wiki(dirname):
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            # 재귀 함수
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)

filepaths = list_wiki('text')

In [27]:
len(filepaths)
# 파일의 개수

958

958개의 파일을 하나로 합친다

In [28]:
with open("output_file.txt", "w") as outfile:
    for filename in filepaths:
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)

파일 읽어서 10줄 출력

In [29]:
f = open('output_file.txt', encoding="utf8")

i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i+1
        print("%d번째 줄 :"%i + line)
    if i==10:
        break 
f.close()

1번째 줄 :<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 :지미 카터

3번째 줄 :제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국의 제39대 대통령(1977년 ~ 1981년) 이다.

4번째 줄 :생애.

5번째 줄 :어린 시절.

6번째 줄 :지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.

7번째 줄 :조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

8번째 줄 :정계 입문.

9번째 줄 :1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.

10번째 줄 :대통령 재임.



## 2. 형태소 분석

In [32]:
from tqdm import tqdm
from konlpy.tag import Mecab

형태소 분석기 Mecab 사용해서 토큰화 진행

In [33]:
mecab = Mecab()

output_file에 몇줄 있는지 확인

In [34]:
f = open('output_file.txt', encoding="utf8")
lines = f.read().splitlines()
print(len(lines))

10888414


In [35]:
lines[:5]
# 상위 5개만 출력

['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">',
 '지미 카터',
 '',
 '제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국의 제39대 대통령(1977년 ~ 1981년) 이다.',
 '생애.']

아무것도 들어있지 않은 줄들은 제외하고 형태소 분석 수행

In [ ]:
result = []

##실행 안됨

for line in tqdm(lines):
  # 빈 문자열이 아닌 경우에만 수행
  if line:
    result.append(mecab.morphs(line))

 32%|███▏      | 3480229/10888414 [30:23<1492:14:21,  1.38it/s]